<a href="https://colab.research.google.com/github/restrepo/articles_COVID-19/blob/master/lens_scotogenic_onodo_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Articles with `COVID-19`  in `title` of Lens.org

See [lens_covid.ipynb](./lens_covid.ipynb) [[GitHub]](https://github.com/restrepo/articles_COVID-19/blob/master/lens_covid.ipynb) [[Colab]](https://colab.research.google.com/github/restrepo/articles_COVID-19/blob/master/lens_covid.ipynb) to get the databases from lens.org API

In [0]:
import pandas as pd
import time

### Articles

Load articles with title covid-19

In [3]:
base='https://github.com/restrepo/articles_COVID-19/raw/master/data/'
file='scotogenic.json.gz'
cv=pd.read_json('{}{}'.format(base,file),compression='gzip')
cv.shape

(1732, 29)

In [0]:
cvac=cv.dropna(subset=['scholarly_citations'])

### Citations

Load citations to articles. Get the metadata for each one of the `lens_id` citations

In [5]:
base='https://github.com/restrepo/articles_COVID-19/raw/master/data/'
file='scotogenic_scholarly_citations.json.gz'
cvc=pd.read_json('{}{}'.format(base,file),compression='gzip')
cvc.shape

(11575, 31)

In [0]:
cvt=cvac.append(cvc).reset_index(drop=True)

In [7]:
cvt.shape

(12721, 31)

In [0]:
cvt=cvt.drop_duplicates('lens_id').reset_index(drop=True)

In [9]:
cvt.shape

(11917, 31)

In [10]:
cv.columns

Index(['lens_id', 'created', 'open_access', 'publication_type', 'authors',
       'title', 'volume', 'issue', 'languages', 'references',
       'fields_of_study', 'source_urls', 'abstract', 'year_published',
       'references_count', 'scholarly_citations_count', 'external_ids',
       'start_page', 'source', 'scholarly_citations', 'author_count',
       'date_published', 'end_page', 'funding',
       'publication_supplementary_type', 'patent_citations',
       'patent_citations_count', 'mesh_terms', 'keywords'],
      dtype='object')

## Onodo network

In [0]:
base='https://github.com/restrepo/articles_COVID-19/raw/master/data/'
file='onodo_scotogenic_SPNL.xlsx'
on1=pd.read_excel('{}{}'.format(base,file),sheet_name=0)
time.sleep(1)
on2=pd.read_excel('{}{}'.format(base,file),sheet_name=1)

In [0]:
on1['Visible']=''
on1['Link']=''
on1['Description']=''

In [13]:
on1[:3]

,Name,Type,Description,Visible,Title,Date,Journal,Lens_ID,Citations,Importance,Link
0,"1-0,8576",Old,,,Radiative Scaling Neutrino Mass and Warm Dark ...,2018-05-15,Physics Letters B,094-690-739-382-440,36,0.8576,
1,"2-0,81",New,,,Light $Z^\prime$ and Dark Matter from U(1)$_X$...,2020-03-23,arXiv: High Energy Physics - Phenomenology,194-666-813-225-917,0,0.8100,
2,"3-0,806",Old,,,Multicomponent dark matter particles in a two-...,2018-05-12,Physical Review D,020-650-282-178-018,69,0.8060,


<div><a href="https://www.lens.org/lens/scholar/article/019-696-870-676-337/main">
Connecting neutrino physics with dark matter</a><br>
Massimiliano Lattanzi , Roberto A Lineros, Marco Taoso <br><br>
The origin of neutrino masses and the nature of dark matter are two in most pressing open questions in modern astro-particle physics. We consider here the possibility that these two problems are related, and review some theoretical scenarios which offer common solutions. ...
<br><br>
DOI: <a href="http://dx.doi.org/10.1088/1367-2630/16/12/125012">10.1088/1367-2630/16/12/125012</a> [<a href="http://arxiv.org/abs/1406.0004">PDF</a>]</div>

In [0]:
from IPython import display

In [0]:
def fill_description(lens_id,title='',authors=None,
                     abstract=None,DOI=None,PDF=None):
    dscr='<div><a href="https://www.lens.org/lens/scholar/article/{}/main">\n'.format(
            lens_id)
    if len(title)>400:
        title=title[:400]+'...'
    dscr=dscr+'{}</a><br>\n'.format(title)
    if authors:    
        dscr=dscr+'{} <br><br>\n'.format(authors)
    if abstract:
        if len(abstract)>400:
            abstract=abstract[:400]+'...'
        dscr=dscr+'{}\n<br><br>\n'.format(abstract)
    if DOI:
        dscr=dscr+'DOI: <a href="http://dx.doi.org/{}">{}</a> '.format(DOI,DOI)
    if PDF:
        dscr=dscr+'[<a href="{}">PDF</a>]</div>'.format(PDF)
    return dscr

In [0]:
def get_metadata(lens_id,df=cvt):
    r=df[df['lens_id']==lens_id].reset_index(drop=True)
    if r.shape[0]==1:
        rr=r.loc[0]
    title=None
    if rr.get('title'):
        title=rr.get('title')
    authors=None
    if rr.get('authors'):
        l=['{} {}'.format(a.get('first_name'),a.get('last_name')) for a in rr.get('authors')]
        etal=''
        if len(l)>3:
            l=l[:3]
            etal=', <i>et al.</i>'
        authors=', '.join(l)+etal
    abstract=None
    if rr.get('abstract'):
        abstract=rr.get('abstract')
    DOI=None
    if rr.get('external_ids'):
        DOIl=[d.get('value') for d in rr.get('external_ids') if d.get('type')=='doi']
        if len(DOIl)>0:
            DOI=DOIl[0]
        
    PDF=None
    if rr.get('source_urls'):
        PDFl=[ d.get('url') for d in rr.get('source_urls') if d.get('url').lower().find('rxiv')>-1 ]
        if len(PDFl)>0:
            PDF=PDFl[0]

    dscr=fill_description(lens_id,title,authors,abstract,DOI,PDF)
    return dscr

In [0]:
on1['Description']=on1['Lens_ID'].apply(lambda s: get_metadata(s) if isinstance(s,str) else '')

In [27]:
display.HTML(on1.sample().reset_index(drop=True).loc[0,'Description'])

In [0]:
from pandas import ExcelWriter
# from pandas.io.parsers import ExcelWriter

In [0]:
with ExcelWriter('scotogenic_covid-19.xlsx') as writer:
    on1.to_excel(writer,'Nodes',index=False)
    on2.to_excel(writer,'Relations',index=False)
    writer.save()

In [0]:
import numpy as np

In [0]:
mask = np.column_stack([cvt[col].astype(str).str.lower().str.contains(r"rxiv", na=False) for col in cvt])
kk=cvt.loc[mask.any(axis=1)][['lens_id','open_access','publication_type','source_urls','external_ids','source']].reset_index(drop=True)

In [0]:
kk

,lens_id,open_access,publication_type,source_urls,external_ids,source
0,019-364-917-998-018,None,None,"[{'type': 'pdf', 'url': 'https://arxiv.org/pdf...","[{'type': 'magid', 'value': '1960786865'}]",{'title': 'arXiv: Populations and Evolution'}
1,029-801-242-386-288,"{'locations': {}, 'colour': 'green'}",journal article,"[{'type': 'html', 'url': 'https://arxiv.org/ab...","[{'type': 'magid', 'value': '2009606198'}, {'t...",{'title': 'Physica A-statistical Mechanics and...
2,000-851-047-428-254,None,preprint,"[{'type': 'html', 'url': 'https://www.biorxiv....","[{'type': 'magid', 'value': '3004151654'}, {'t...",{'title': 'bioRxiv'}
3,002-281-978-092-075,None,preprint,"[{'type': 'unknown', 'url': 'https://www.medrx...","[{'type': 'magid', 'value': '3006592697'}, {'t...",{'title': 'medRxiv'}
4,002-906-736-392-056,None,preprint,"[{'type': 'html', 'url': 'https://www.medrxiv....","[{'type': 'magid', 'value': '3006590857'}, {'t...",{'title': 'medRxiv'}
...,...,...,...,...,...,...
535,139-143-876-442-831,None,preprint,"[{'type': 'html', 'url': 'https://europepmc.or...","[{'type': 'magid', 'value': '3008897748'}, {'t...",{'title': 'bioRxiv'}
536,010-093-802-970-403,None,preprint,"[{'type': 'html', 'url': 'https://www.medrxiv....","[{'type': 'magid', 'value': '3010211880'}, {'t...",{'title': 'medRxiv'}
537,000-071-000-180-100,None,None,"[{'type': 'html', 'url': 'https://arxiv.org/pd...","[{'type': 'magid', 'value': '3009762978'}]",{'title': 'arXiv: Physics and Society'}
538,042-986-203-395-32X,None,preprint,"[{'type': 'html', 'url': 'https://www.medrxiv....","[{'type': 'magid', 'value': '3009526522'}, {'t...",{'title': 'medRxiv'}


In [0]:
kk

,open_access,publication_type,source_urls,external_ids,source
0,None,preprint,"[{'type': 'unknown', 'url': 'https://www.medrx...","[{'type': 'magid', 'value': '3006592697'}, {'t...",{'title': 'medRxiv'}


In [0]:
kk.loc[0,'external_ids']

[{'type': 'magid', 'value': '3006592697'},
 {'type': 'doi', 'value': '10.1101/2020.02.12.20022327'}]

In [0]:
cvt[cvt.external_ids.astype(str).str.lower().str.contains('xiv')]['external_ids']

1026    [{'type': 'doi', 'value': '10.26434/chemrxiv.1...
1236    [{'type': 'doi', 'value': '10.26434/chemrxiv.1...
Name: external_ids, dtype: object